Building A Question-Answering System With LlamaIndex


In [11]:
!pip install --upgrade llama-index
!pip install llama-index-llms-huggingface
!pip install llama-index-llms-huggingface-api
!pip install llama-index transformers accelerate bitsandbytes sentencepiece
!pip install transformers

In [15]:
# Import necessary classes and functions 
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, ServiceContext, StorageContext, load_index_from_storage, Settings
from llama_index.core import StorageContext, load_index_from_storage
from llama_index.llms.huggingface import HuggingFaceLLM
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

In [16]:
# Load the token HF
from dotenv import load_dotenv
import os

load_dotenv()  # Charge les variables du .env
hf_token = os.getenv("HUGGING_FACE_HUB_TOKEN")

In [17]:
# Text.pdf
!pip install pymupdf
import fitz  # PyMuPDF

pdf_path = '/Users/patash/PSTB/Week_5_NLP/day_5/Mini_project/230714334.pdf'

def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page in doc:
        text += page.get_text() + "\n"  # Récupère le texte de chaque page
    return text

text = extract_text_from_pdf(pdf_path)

print(text[:1000])  

Towards Generalist Biomedical AI
Tao Tu∗, ‡, 1, Shekoofeh Azizi∗, ‡, 2,
Danny Driess2, Mike Schaekermann1, Mohamed Amin1, Pi-Chuan Chang1, Andrew Carroll1,
Chuck Lau1, Ryutaro Tanno2, Ira Ktena2, Basil Mustafa2, Aakanksha Chowdhery2, Yun Liu1,
Simon Kornblith2, David Fleet2, Philip Mansfield1, Sushant Prakash1, Renee Wong1, Sunny Virmani1,
Christopher Semturs1, S Sara Mahdavi2, Bradley Green1, Ewa Dominowska1, Blaise Aguera y Arcas1,
Joelle Barral2, Dale Webster1, Greg S. Corrado1, Yossi Matias1, Karan Singhal1, Pete Florence2,
Alan Karthikesalingam†, ‡,1 and Vivek Natarajan†, ‡,1
1Google Research, 2Google DeepMind
Medicine is inherently multimodal, with rich data modalities spanning text, imaging, genomics, and more.
Generalist biomedical artificial intelligence (AI) systems that flexibly encode, integrate, and interpret
this data at scale can potentially enable impactful applications ranging from scientific discovery to care
delivery. To enable the development of these models, we fir

In [18]:
# Initilize LLM
# Charger le tokenizer et le modèle
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-large")
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-large")

# Initialiser le modèle LLM avec HuggingFace
hf_llm = HuggingFaceLLM(
    model_name="google/flan-t5-large", 
    tokenizer_name="google/flan-t5-large", 
    context_window=2048,  # Taille de la fenêtre de contexte
    max_new_tokens=256,  # Nombre maximal de tokens à générer
    model_kwargs={"temperature": 0.1, "max_length": 512},  # Paramètres de génération
    device_map="auto",  # Laisser Hugging Face choisir automatiquement l'appareil (GPU ou CPU)
    model=model,  # Charger le modèle
    tokenizer=tokenizer  # Charger le tokenizer
)


llm = HuggingFaceLLM(model = model, tokenizer=tokenizer, context_window=2048,
    max_new_tokens=256, model_kwargs={"temperature": 0.1, "max_length": 512}, device_map="auto")

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [20]:
!pip install llama-index-embeddings-huggingface
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

hf_embedding = HuggingFaceEmbedding(
    model_name="sentence-transformers/all-MiniLM-L6-v2",  # Exemple de modèle d'embedding
    model_kwargs={"device": "cuda"}  # Utiliser le GPU si disponible
)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

FileNotFoundError: [Errno 2] No such file or directory: 'sentence-transformers/all-MiniLM-L6-v2/sentence_xlnet_config.json'